In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores.faiss import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter(separator="\n", chunk_size=600, chunk_overlap=100)

loader = UnstructuredFileLoader("./files/AP_01.pdf")


docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings() 

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir, namespace=embeddings.model)

vectorstore = FAISS.from_documents(docs, cache_embeddings)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("How many ministries are mentioned")

AIMessage(content='사회적 지능을 높이기 위해서는 다음과 같은 방법들을 고려할 수 있습니다:\n\n1. 커뮤니케이션 능력 향상: 효과적인 대화 기술과 비언어적 신호 해석 능력을 향상시키는 것이 중요합니다. 다른 사람들과의 원활한 대화를 위해 리스닝 및 의사소통 기술을 개발하고, 상황에 맞는 언어와 태도를 사용하는 방법을 익히는 것이 도움이 될 수 있습니다.\n\n2. 유머 감각 개발: 유머 감각은 사회적 상황에서 긍정적인 인상을 주는 데 도움이 됩니다. 유머 감각을 향상시키기 위해 다양한 유머 스타일과 문화를 이해하고, 적절한 타이밍과 상황에서 유머를 사용하는 방법을 연습할 수 있습니다.\n\n3. 친화력 강화: 다른 사람들과의 관계를 발전시키기 위해 친화력을 향상시키는 것이 중요합니다. 다른 사람들과의 연결을 증진시키기 위해 관심을 가지고 리스닝하고, 상대방의 감정을 이해하고 공감하는 능력을 키울 수 있습니다.\n\n4. 협상력 향상: 협상은 사회적 상황에서 중요한 요소입니다. 상대방의 요구와 자신의 목표를 조율하고, 상황에 맞는 해결책을 찾는 능력을 향상시키는 것이 도움이 됩니다. 협상 기술을 배우고 연습하여 상호 이익을 극대화할 수 있습니다.\n\n5. 전달력 강화: 명확하고 효과적인 메시지 전달은 사회적 지능을 향상시키는 데 도움이 됩니다. 목적에 맞는 언어와 커뮤니케이션 스타일을 사용하여 아이디어와 의견을 효과적으로 전달하는 방법을 연습할 수 있습니다.\n\n이러한 방법들을 통해 사회적 지능을 향상시킬 수 있지만, 중요한 것은 지속적인 학습과 연습입니다. 일상적인 상황에서 이러한 기술들을 적용하고 발전시키는 것이 중요합니다.')